In [1]:
import requests
import lxml.html as lh
import pandas as pd
import numpy as np

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'#Create a handle, page, to handle the contents of the website
page = requests.get(url)#Store the contents of the website under doc
doc = lh.fromstring(page.content)#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [3]:
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [4]:
tr_elements = doc.xpath('//tr')#Create empty list
col=[]
i=0#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print (i,name)
    col.append((name,[]))

1 Postcode
2 Borough
3 Neighbourhood



In [5]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
            
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [6]:
[len(C) for (title,C) in col]

[287, 287, 287]

In [7]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [8]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


In [9]:
df = df.replace(r'\n',' ', regex=True)
df.columns = ['Postcode', 'Borough', 'Neighbourhood']
df.columns

Index(['Postcode', 'Borough', 'Neighbourhood'], dtype='object')

In [10]:
df = df[~df.Borough.str.contains("Not assigned")]
df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df['Neighbourhood'] = np.where(df.Neighbourhood.str.contains ('Not assigned'), df.Borough, df.Neighbourhood)
df.head()


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge , Malvern"
1,M1C,Scarborough,"Highland Creek , Rouge Hill , Port Union"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [11]:
df.shape

(103, 3)

In [12]:
!wget -O latlongs http://cocl.us/Geospatial_data

--2020-02-11 20:50:49--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.48.113.194, 158.85.108.86, 158.85.108.83
Connecting to cocl.us (cocl.us)|169.48.113.194|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2020-02-11 20:50:49--  https://cocl.us/Geospatial_data
Connecting to cocl.us (cocl.us)|169.48.113.194|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-02-11 20:50:52--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.27.197, 107.152.26.197
Connecting to ibm.box.com (ibm.box.com)|107.152.27.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-02-11 20:50:52--  https://

In [13]:
dfll = pd.read_csv('latlongs')
dfll.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
dat1 = pd.concat([df, dfll], axis=1)
dat1.drop('Postal Code', axis = 1)
dat1

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge , Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek , Rouge Hill , Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,M1J,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park , Ionview , Kennedy Park",M1K,43.727929,-79.262029
7,M1L,Scarborough,"Clairlea , Golden Mile , Oakridge",M1L,43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest , Cliffside , Scarborough Village W...",M1M,43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff , Cliffside West",M1N,43.692657,-79.264848


In [15]:
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [16]:
!conda install -c conda-forge geopy --yes 

Solving environment: done

# All requested packages already installed.



In [17]:
address = 'Toronto'

geolocator = Nominatim(user_agent="TR_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [18]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(dat1['Latitude'], dat1['Longitude'], dat1['Borough'], dat1['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [19]:
CLIENT_ID = 'LWH4BTBD0GXHMNHZZAF1VJEGOEMZ5PNYQIKWLRGN5AC0PZ2S' # your Foursquare ID
CLIENT_SECRET = 'DXKDNAV43FJOY0MF4DLRRMZBN0OOXFTGHZJHHFYGZ5GO3OQZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LWH4BTBD0GXHMNHZZAF1VJEGOEMZ5PNYQIKWLRGN5AC0PZ2S
CLIENT_SECRET:DXKDNAV43FJOY0MF4DLRRMZBN0OOXFTGHZJHHFYGZ5GO3OQZ


In [20]:
dat1.loc[0, 'Neighbourhood']

'Rouge , Malvern '

In [21]:
neighbourhood_latitude = dat1.loc[0, 'Latitude'] # neighbourhood latitude value
neighbourhood_longitude = dat1.loc[0, 'Longitude'] # neighbourhood longitude value

neighbourhood_name = dat1.loc[0, 'Neighbourhood'] # neighbourhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Rouge , Malvern  are 43.806686299999996, -79.19435340000001.


In [22]:
# Foursquare request URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    750, 
    100)
url




'https://api.foursquare.com/v2/venues/explore?&client_id=LWH4BTBD0GXHMNHZZAF1VJEGOEMZ5PNYQIKWLRGN5AC0PZ2S&client_secret=DXKDNAV43FJOY0MF4DLRRMZBN0OOXFTGHZJHHFYGZ5GO3OQZ&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=750&limit=100'

In [23]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e4315380be7b4001bc2025d'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 8,
  'suggestedBounds': {'ne': {'lat': 43.81343630675, 'lng': -79.18501767109562},
   'sw': {'lat': 43.79993629324999, 'lng': -79.2036891289044}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '579a91b3498e9bd833afa78a',
       'name': "Wendy's",
       'location': {'address': '8129 Sheppard Avenue',
        'lat': 43.8020084,
        'lng': -79.1980797,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.8020084,
          'lng': -79.19

In [24]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [25]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy's,Fast Food Restaurant,43.802008,-79.198080
1,Wendy's,Fast Food Restaurant,43.807448,-79.199056
2,Staples Morningside,Paper / Office Supplies Store,43.800285,-79.196607
3,Tim Hortons,Coffee Shop,43.802000,-79.198169
4,Lee Valley,Hobby Shop,43.803161,-79.199681


In [26]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

8 venues were returned by Foursquare.


In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            500, 
            100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [29]:
# type your answer here

toronto_venues = getNearbyVenues(names=dat1['Neighbourhood'],
                                   latitudes=dat1['Latitude'],
                                   longitudes=dat1['Longitude']
                                  )
toronto_venues.head()


Rouge , Malvern 
Highland Creek , Rouge Hill , Port Union 
Guildwood , Morningside , West Hill 
Woburn 
Cedarbrae 
Scarborough Village 
East Birchmount Park , Ionview , Kennedy Park 
Clairlea , Golden Mile , Oakridge 
Cliffcrest , Cliffside , Scarborough Village West 
Birch Cliff , Cliffside West 
Dorset Park , Scarborough Town Centre , Wexford Heights 
Maryvale , Wexford 
Agincourt 
Clarks Corners , Sullivan , Tam O'Shanter 
Agincourt North , L'Amoreaux East , Milliken , Steeles East 
L'Amoreaux West 
Upper Rouge 
Hillcrest Village 
Fairview , Henry Farm , Oriole 
Bayview Village 
Silver Hills , York Mills 
Newtonbrook , Willowdale 
Willowdale South 
York Mills West 
Willowdale West 
Parkwoods 
Don Mills North 
Flemingdon Park , Don Mills South 
Bathurst Manor , Downsview North , Wilson Heights 
Northwood Park , York University 
CFB Toronto , Downsview East 
Downsview West 
Downsview Central 
Downsview Northwest 
Victoria Village 
Woodbine Gardens , Parkview Hill 
Woodbine Heights 
Th

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge , Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge , Malvern",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Highland Creek , Rouge Hill , Port Union",43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
3,"Highland Creek , Rouge Hill , Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
4,"Highland Creek , Rouge Hill , Port Union",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target


In [30]:
print(toronto_venues.shape)
toronto_venues.head()

(2213, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge , Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge , Malvern",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Highland Creek , Rouge Hill , Port Union",43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
3,"Highland Creek , Rouge Hill , Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
4,"Highland Creek , Rouge Hill , Port Union",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target


In [31]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide , King , Richmond",100,100,100,100,100,100
Agincourt,5,5,5,5,5,5
"Agincourt North , L'Amoreaux East , Milliken , Steeles East",2,2,2,2,2,2
"Albion Gardens , Beaumond Heights , Humbergate , Jamestown , Mount Olive , Silverstone , South Steeles , Thistletown",10,10,10,10,10,10
"Alderwood , Long Branch",9,9,9,9,9,9
"Bathurst Manor , Downsview North , Wilson Heights",21,21,21,21,21,21
Bayview Village,4,4,4,4,4,4
"Bedford Park , Lawrence Manor East",25,25,25,25,25,25
Berczy Park,55,55,55,55,55,55


In [32]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 266 uniques categories.


In [33]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Rouge , Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Rouge , Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Highland Creek , Rouge Hill , Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Highland Creek , Rouge Hill , Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Highland Creek , Rouge Hill , Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
toronto_onehot.shape

(2213, 267)

In [35]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide , King , Richmond",0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.020000,...,0.0,0.020000,0.00,0.000000,0.000000,0.000000,0.010000,0.0,0.01,0.000000
1,Agincourt,0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000
2,"Agincourt North , L'Amoreaux East , Milliken ,...",0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000
3,"Albion Gardens , Beaumond Heights , Humbergate...",0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000
4,"Alderwood , Long Branch",0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000
5,"Bathurst Manor , Downsview North , Wilson Heig...",0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.0,0.000000,0.00,0.047619,0.000000,0.000000,0.000000,0.0,0.00,0.000000
6,Bayview Village,0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000
7,"Bedford Park , Lawrence Manor East",0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.040000,...,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000
8,Berczy Park,0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.0,0.018182,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000
9,"Birch Cliff , Cliffside West",0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000


In [36]:
toronto_grouped.shape

(101, 267)

In [37]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide , King , Richmond ----
             venue  freq
0      Coffee Shop  0.07
1              Bar  0.04
2             Café  0.04
3       Steakhouse  0.04
4  Thai Restaurant  0.04


----Agincourt ----
                       venue  freq
0                     Lounge   0.2
1             Breakfast Spot   0.2
2  Latin American Restaurant   0.2
3             Clothing Store   0.2
4               Skating Rink   0.2


----Agincourt North , L'Amoreaux East , Milliken , Steeles East ----
                        venue  freq
0                        Park   0.5
1                  Playground   0.5
2           Accessories Store   0.0
3               Metro Station   0.0
4  Modern European Restaurant   0.0


----Albion Gardens , Beaumond Heights , Humbergate , Jamestown , Mount Olive , Silverstone , South Steeles , Thistletown ----
                 venue  freq
0        Grocery Store   0.2
1          Pizza Place   0.1
2  Fried Chicken Joint   0.1
3          Coffee Shop   0.1
4       Sandwich Place 

                        venue  freq
0                Home Service  0.33
1              Baseball Field  0.33
2                  Food Truck  0.33
3         Monument / Landmark  0.00
4  Modern European Restaurant  0.00


----Downsview Northwest ----
                  venue  freq
0  Gym / Fitness Center  0.25
1          Liquor Store  0.25
2         Grocery Store  0.25
3    Athletics & Sports  0.25
4     Accessories Store  0.00


----Downsview West ----
           venue  freq
0           Bank  0.25
1           Park  0.25
2  Grocery Store  0.25
3  Shopping Mall  0.25
4  Metro Station  0.00


----East Birchmount Park , Ionview , Kennedy Park ----
                venue  freq
0    Department Store  0.25
1  Chinese Restaurant  0.25
2          Hobby Shop  0.25
3         Coffee Shop  0.25
4  Miscellaneous Shop  0.00


----East Toronto ----
                venue  freq
0                Park  0.33
1         Coffee Shop  0.33
2   Convenience Store  0.33
3   Accessories Store  0.00
4  Mexican Restauran

                     venue  freq
0                   Garden  0.33
1  Health & Beauty Service  0.33
2                     Pool  0.33
3        Accessories Store  0.00
4       Mexican Restaurant  0.00


----Rouge , Malvern ----
                             venue  freq
0             Fast Food Restaurant   0.5
1                       Print Shop   0.5
2               Mexican Restaurant   0.0
3  Molecular Gastronomy Restaurant   0.0
4       Modern European Restaurant   0.0


----Runnymede , Swansea ----
                venue  freq
0                Café  0.08
1         Coffee Shop  0.08
2         Pizza Place  0.08
3    Sushi Restaurant  0.05
4  Italian Restaurant  0.05


----Ryerson , Garden District ----
                 venue  freq
0          Coffee Shop  0.09
1       Clothing Store  0.08
2                 Café  0.03
3  Japanese Restaurant  0.03
4       Cosmetics Shop  0.03


----Scarborough Village ----
                             venue  freq
0                       Playground   1.0
1     

In [38]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [39]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide , King , Richmond",Coffee Shop,Steakhouse,Thai Restaurant,Café,Bar,Burger Joint,Cosmetics Shop,Bakery,Breakfast Spot,Asian Restaurant
1,Agincourt,Latin American Restaurant,Skating Rink,Lounge,Clothing Store,Breakfast Spot,Drugstore,Discount Store,Dog Run,Doner Restaurant,Donut Shop
2,"Agincourt North , L'Amoreaux East , Milliken ,...",Park,Playground,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
3,"Albion Gardens , Beaumond Heights , Humbergate...",Grocery Store,Fried Chicken Joint,Pizza Place,Fast Food Restaurant,Beer Store,Sandwich Place,Liquor Store,Coffee Shop,Pharmacy,Comic Shop
4,"Alderwood , Long Branch",Pizza Place,Sandwich Place,Gym,Coffee Shop,Skating Rink,Pub,Pharmacy,Pool,Yoga Studio,Dessert Shop


In [40]:
# set number of clusters
kclusters = 7

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_


array([0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 2, 4, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 6, 3, 0, 0, 0, 0, 2, 0, 6, 0, 5, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2], dtype=int32)

In [41]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = dat1

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
toronto_merged.head() # check the last columns!
toronto_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 17 columns):
Postcode                  103 non-null object
Borough                   103 non-null object
Neighbourhood             103 non-null object
Postal Code               103 non-null object
Latitude                  103 non-null float64
Longitude                 103 non-null float64
Cluster Labels            101 non-null float64
1st Most Common Venue     101 non-null object
2nd Most Common Venue     101 non-null object
3rd Most Common Venue     101 non-null object
4th Most Common Venue     101 non-null object
5th Most Common Venue     101 non-null object
6th Most Common Venue     101 non-null object
7th Most Common Venue     101 non-null object
8th Most Common Venue     101 non-null object
9th Most Common Venue     101 non-null object
10th Most Common Venue    101 non-null object
dtypes: float64(3), object(14)
memory usage: 13.8+ KB


In [42]:

toronto_merged = toronto_merged.fillna(0) # replace nan cells with integer 0

toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int) 

In [43]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [52]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Highland Creek , Rouge Hill , Port Union",-79.160497,0,Bar,Moving Target,Construction & Landscaping,Yoga Studio,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
2,"Guildwood , Morningside , West Hill",-79.188711,0,Medical Center,Intersection,Electronics Store,Breakfast Spot,Pizza Place,Mexican Restaurant,Rental Car Location,Dim Sum Restaurant,Diner,Discount Store
3,Woburn,-79.216917,0,Coffee Shop,Korean Restaurant,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
4,Cedarbrae,-79.239476,0,Bakery,Hakka Restaurant,Fried Chicken Joint,Caribbean Restaurant,Athletics & Sports,Thai Restaurant,Gas Station,Bank,Dog Run,Dim Sum Restaurant
6,"East Birchmount Park , Ionview , Kennedy Park",-79.262029,0,Hobby Shop,Coffee Shop,Chinese Restaurant,Department Store,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dessert Shop
7,"Clairlea , Golden Mile , Oakridge",-79.284577,0,Bakery,Bus Line,Park,Ice Cream Shop,Soccer Field,Bus Station,Intersection,Doner Restaurant,Discount Store,Dog Run
8,"Cliffcrest , Cliffside , Scarborough Village W...",-79.239476,0,American Restaurant,Motel,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
9,"Birch Cliff , Cliffside West",-79.264848,0,College Stadium,Café,General Entertainment,Skating Rink,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore
10,"Dorset Park , Scarborough Town Centre , Wexfor...",-79.273304,0,Indian Restaurant,Chinese Restaurant,Pet Store,Vietnamese Restaurant,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
11,"Maryvale , Wexford",-79.295849,0,Middle Eastern Restaurant,Bakery,Breakfast Spot,Smoke Shop,Yoga Studio,Donut Shop,Discount Store,Dog Run,Doner Restaurant,Dumpling Restaurant


In [53]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough Village,-79.239476,1,Playground,Yoga Studio,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore


In [54]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,York Mills West,-79.400049,2,Park,Bank,Convenience Store,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dessert Shop,Drugstore
25,Parkwoods,-79.329656,2,Park,Food & Drink Shop,Department Store,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop
40,East Toronto,-79.338106,2,Park,Convenience Store,Coffee Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
74,Caledonia-Fairbanks,-79.453512,2,Park,Fast Food Restaurant,Market,Women's Store,Golf Course,Deli / Bodega,Greek Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
90,"The Kingsway , Montgomery Road , Old Mill North",-79.506944,2,Park,River,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Falafel Restaurant,Donut Shop
98,Weston,-79.518188,2,Park,Convenience Store,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Department Store,Drugstore


In [55]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,"Newtonbrook , Willowdale",-79.408493,3,Gym,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore


In [56]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Downsview Central,-79.495697,4,Baseball Field,Food Truck,Home Service,Yoga Studio,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
91,"Humber Bay , King's Mill Park , Kingsway Park ...",-79.498509,4,Baseball Field,Home Service,Yoga Studio,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,Dessert Shop
97,"Emery , Humberlea",-79.532242,4,Baseball Field,Construction & Landscaping,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant


In [57]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Rouge , Malvern",-79.194353,5,Fast Food Restaurant,Print Shop,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore


In [58]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 6, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,"Agincourt North , L'Amoreaux East , Milliken ,...",-79.284577,6,Park,Playground,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
48,"Moore Park , Summerhill East",-79.383160,6,Park,Playground,Tennis Court,Restaurant,Eastern European Restaurant,Dumpling Restaurant,Electronics Store,Drugstore,Donut Shop,Deli / Bodega
50,Rosedale,-79.377529,6,Park,Trail,Playground,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop


In [ ]:
# Amalysis:
# Cluster 0: Dining areas
# Cluster 1: Scarborough Village, only 1 data point.
# Cluster 2: Park + shopping/dining areas
# Cluster 3: Newtonbrook , Willowdale, only 1 data point, gym area?
# Cluster 4: Baseball feilds / activity area
# Cluster 5: Rouge , Malvern, only 1 data point
# Cluster 6: Park/Outdoor areas.